In [8]:
from experta import *
from dataclasses import dataclass
from typing import Optional
from enum import Enum
import random
import time
from datetime import datetime

class TimeOfDay(Enum):
    DAY = "day"
    NIGHT = "night"

@dataclass
class EnvironmentState:
    temperature: float
    humidity: float
    light_level: int
    motion_detected: bool
    time: TimeOfDay
    
    @classmethod
    def generate_random(cls):
        """Генерує випадковий стан середовища"""
        # Визначаємо час доби на основі реального часу
        current_hour = datetime.now().hour
        time_of_day = TimeOfDay.DAY if 6 <= current_hour < 22 else TimeOfDay.NIGHT
        
        return cls(
            temperature=round(random.uniform(15, 35), 1),
            humidity=round(random.uniform(30, 90), 1),
            light_level=random.randint(0, 100),
            motion_detected=random.choice([True, False]),
            time=time_of_day
        )
    
    def to_facts(self) -> list:
        """Конвертує стан середовища у факти для експертної системи"""
        return [
            Fact(temperature=self.temperature),
            Fact(humidity=self.humidity),
            Fact(light=self.light_level),
            Fact(motion=self.motion_detected),
            Fact(time=self.time.value)
        ]
    
    def __str__(self):
        return (f"\nСтан середовища:"
                f"\n- Температура: {self.temperature}°C"
                f"\n- Вологість: {self.humidity}%"
                f"\n- Рівень освітлення: {self.light_level}"
                f"\n- Рух: {'Виявлено' if self.motion_detected else 'Не виявлено'}"
                f"\n- Час доби: {self.time.value}")

@dataclass
class SystemState:
    """Стан виконавчих механізмів системи"""
    ac_enabled: bool = False
    heating_enabled: bool = False
    lights_enabled: bool = False
    ventilation_enabled: bool = False

class HomeAutomationSystem(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.system_state = SystemState()
    
    def update_system_state(self, **kwargs):
        """Оновлює стан системи та логує зміни"""
        for key, value in kwargs.items():
            if hasattr(self.system_state, key):
                old_value = getattr(self.system_state, key)
                setattr(self.system_state, key, value)
                if old_value != value:
                    print(f"Змінено {key}: {old_value} -> {value}")
    
    @Rule(Fact(temperature=P(lambda x: x > 30)))
    def turn_on_ac(self):
        self.update_system_state(
            ac_enabled=True,
            heating_enabled=False
        )
        print(f"Температура висока ({self.facts[1]['temperature']}°C) - вмикаю кондиціонер")
    
    @Rule(Fact(temperature=P(lambda x: x < 20)))
    def turn_on_heating(self):
        self.update_system_state(
            heating_enabled=True,
            ac_enabled=False
        )
        print(f"Температура низька ({self.facts[1]['temperature']}°C) - вмикаю обігрів")
    
    @Rule(AND(
        Fact(time='night'),
        Fact(motion=True),
        Fact(light=P(lambda x: x < 10))
    ))
    def turn_on_lights(self):
        self.update_system_state(lights_enabled=True)
        print("Виявлено рух вночі при низькому освітленні - вмикаю світло")
    
    @Rule(Fact(humidity=P(lambda x: x > 70)))
    def turn_on_ventilation(self):
        self.update_system_state(ventilation_enabled=True)
        print(f"Висока вологість ({self.facts[2]['humidity']}%) - вмикаю вентиляцію")
    
    @Rule(AND(
        Fact(time='day'),
        Fact(light=P(lambda x: x > 50)),
        Fact(motion=False)
    ))
    def energy_saving(self):
        self.update_system_state(lights_enabled=False)
        print("Денне світло, рух відсутній - вимикаю освітлення для економії")

def run_system_with_timer(duration_seconds=30):
    print(f"Запуск системи на {duration_seconds} секунд...")
    
    engine = HomeAutomationSystem()
    start_time = time.time()
    iteration = 1
    
    try:
        while time.time() - start_time < duration_seconds:
            print(f"\n=== Ітерація {iteration} ===")
            # Генеруємо новий випадковий стан
            current_state = EnvironmentState.generate_random()
            print(current_state)
            
            # Скидаємо двигун та додаємо нові факти
            engine.reset()
            for fact in current_state.to_facts():
                engine.declare(fact)
            
            # Запускаємо правила
            engine.run()
            
            # Виводимо поточний стан системи
            print("\nПоточний стан системи:")
            for key, value in engine.system_state.__dict__.items():
                print(f"{key}: {value}")
            
            iteration += 1
            time.sleep(5)  # Чекаємо 5 секунд перед наступною ітерацією
            
    except KeyboardInterrupt:
        print("\nРоботу системи перервано користувачем")
    
    print("\nРоботу системи завершено")

if __name__ == "__main__":
    run_system_with_timer(30) 

Запуск системи на 30 секунд...

=== Ітерація 1 ===

Стан середовища:
- Температура: 25.5°C
- Вологість: 34.4%
- Рівень освітлення: 97
- Рух: Виявлено
- Час доби: day

Поточний стан системи:
ac_enabled: False
heating_enabled: False
lights_enabled: False
ventilation_enabled: False

=== Ітерація 2 ===

Стан середовища:
- Температура: 15.6°C
- Вологість: 82.0%
- Рівень освітлення: 28
- Рух: Не виявлено
- Час доби: day
Змінено ventilation_enabled: False -> True
Висока вологість (82.0%) - вмикаю вентиляцію
Змінено heating_enabled: False -> True
Температура низька (15.6°C) - вмикаю обігрів

Поточний стан системи:
ac_enabled: False
heating_enabled: True
lights_enabled: False
ventilation_enabled: True

=== Ітерація 3 ===

Стан середовища:
- Температура: 30.9°C
- Вологість: 57.7%
- Рівень освітлення: 66
- Рух: Виявлено
- Час доби: day
Змінено ac_enabled: False -> True
Змінено heating_enabled: True -> False
Температура висока (30.9°C) - вмикаю кондиціонер

Поточний стан системи:
ac_enabled: True
